In [1]:
import pandas as pd
df = pd.read_csv('top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/Users/Aayush/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/Users/Aayush/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/cd/lwb11qjn7jq55cw_3z211swc0000gp/T/ipykernel_43130/3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s ability to defer immediate gratification. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge in popularity of Malbec in recent year

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [10]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ChatCompletionMessage(content='I recommend the Catena Zapata Adrianna Vineyard Malbec 2004 from Argentina. It has a rating of 97 and is described as having a complex perfume, layers of fruit, and a long, fruit-filled finish. It is a full-bodied wine that will age effortlessly for another 6-8 years and provide pleasure through 2025. Catena Zapata is the standard of excellence for comparing all others and is an essential destination for fans of both architecture and wine in Mendoza.</s>', role='assistant', function_call=None, tool_calls=None)
